<a href="https://colab.research.google.com/github/huavinhkhang0405/TriTueNhanTao/blob/main/BaoCao_THTTNT_Buoi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MINIMAX

---



*1. Định nghĩa và mục đích*

---



- Minimax là một thuật toán tìm kiếm đệ quy áp dụng cho các trò chơi đối kháng hai người. Nó vận hành bằng cách xây dựng và định giá trị cho các nút (node) trên cây trò chơi, từ đó xác định nước đi tiếp theo tối ưu nhất.
- Minimax là một phương pháp trong lý thuyết quyết định hướng tới việc tối thiểu hóa (Minimize) mức thiệt hại tối đa (Maximum) có thể xảy ra.

*2. Cơ chế hoạt động*

---



Quá trình thực thi của Minimax có thể được mô hình hóa qua 3 giai đoạn tuần tự:
1.  **Duyệt cây và Mở rộng không gian trạng thái**: Thuật toán thực hiện tìm kiếm theo chiều sâu (Depth-First Search - DFS) trên cây trò chơi. Từ trạng thái hiện tại (nút gốc), thuật toán sinh ra các trạng thái kế tiếp cho đến khi chạm tới độ sâu giới hạn hoặc đạt tới các trạng thái kết thúc (nút lá/terminal nodes).
2. **Định giá**: Tại các nút lá, thuật toán áp dụng một hàm định giá để gán một giá trị cho trạng thái đó. Giá trị này đại diện cho lợi ích của người chơi MAX tại thời điểm kết thúc (VD: +1 cho thắng, -1 cho thua, 0 cho hòa).
3. **Lan truyền ngược và Tối ưu hóa cục bộ**: Các giá trị từ nút lá được truyền ngược lên các nút cha (bottom-up approach) theo nguyên tắc đối ngẫu:
*  **Tại lớp MAX**: Giá trị của nút cha được xác định là giá trị lớn nhất trong các nút con (tối đa hóa lợi ích).
*  **Tại lớp MIN**: Giá trị của nút cha được xác định là giá trị nhỏ nhất trong các nút con (giả định đối thủ chơi tối ưu để giảm thiểu lợi ích của MAX).


*3. Ví dụ minh họa - Trò chơi cờ caro (TicTacToe)*

---

Trong ngữ cảnh trò chơi Cờ Caro tổng quát (bao gồm Tic-Tac-Toe và Gomoku), Minimax là giải thuật tìm kiếm nước đi tối ưu trong không gian trạng thái.
- Đối với bàn cờ nhỏ ($3 \times 3$): Thuật toán có khả năng duyệt toàn bộ cây trò
chơi (full state-space search) đến tận trạng thái kết thúc để đưa ra quyết định chính xác tuyệt đối.
- Đối với bàn cờ lớn ($N \times N$): Do sự bùng nổ tổ hợp (combinatorial explosion), thuật toán buộc phải giới hạn độ sâu tìm kiếm (depth-limited search). Khi đó, Minimax sử dụng hàm lượng giá Heuristic để ước lượng tiềm năng của các trạng thái trung gian thay vì đi đến trạng thái kết thúc thực sự.

*Khởi tạo và Cấu hình*

---

Khai báo thư viện và các biến toàn cục đại diện cho người chơi và trạng thái ô trống.

In [34]:
import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

*Các hàm xử lý trạng thái*

---
Nhóm hàm này chịu trách nhiệm khởi tạo bàn cờ, xác định lượt đi, tìm các nước đi hợp lệ và cập nhật bàn cờ sau khi đi.


In [35]:
# Hàm khởi tạo trạng thái bàn cờ ban đầu
def initial_state(n):
    """
    Khởi tạo và trả về bàn cờ trống kích thước n x n.
    """
    return [[EMPTY for _ in range(n)] for _ in range(n)]

# Hàm xác định lượt người chơi hiện tại
def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo dựa trên trạng thái bàn cờ.
    - Đếm số quân cờ đã đánh để xác định lượt (chẵn là người chơi, lẻ là máy).
    """
    count = 0
    for i in board:
        for j in i:
            if j: # Nếu ô không phải là None (đã có người đánh)
                count += 1

    # Nếu số nước đi là lẻ -> Lượt của máy (người đi sau)
    if count % 2 != 0:
        return ai
    # Nếu số nước đi là chẵn -> Lượt của người chơi (người đi trước)
    return user

# Hàm lấy danh sách các nước đi hợp lệ
def actions(board):
    """
    Trả về tập hợp tất cả các nước đi khả thi (i, j) trên bàn cờ.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

# Hàm trả về trạng thái bàn cờ mới (Transition Model)
def result(board, action):
    """
    Trả về trạng thái bàn cờ MỚI sau khi thực hiện nước đi (i, j).
    """
    curr_player = player(board)       # Xác định xem ai đang đi nước này
    result_board = copy.deepcopy(board) # Tạo bản sao bàn cờ
    (i, j) = action
    result_board[i][j] = curr_player  # Đánh dấu nước đi
    return result_board

*Các hàm kiểm tra kết quả và luật chơi*

---

Nhóm hàm này kiểm tra xem ai thắng (ngang, dọc, chéo), game đã kết thúc chưa (terminal) và chấm điểm bàn cờ (utility).

In [36]:
# Hàm kiểm tra thắng theo hàng ngang
def get_horizontal_winner(board):
    """
    Kiểm tra xem có người thắng theo các hàng ngang hay không.
    Trả về ký tự người thắng ('X' hoặc 'O') hoặc None.
    """
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[i][0] # Lấy ô đầu tiên của hàng làm mẫu
        for j in range(board_len):
            if board[i][j] != winner_val: # Nếu có ô khác với ô mẫu -> Hàng này không thắng
                winner_val = None
        if winner_val: # Nếu winner_val vẫn khác None và khác False -> Tìm thấy người thắng
            return winner_val
    return winner_val

# Hàm kiểm tra thắng theo hàng dọc
def get_vertical_winner(board):
    """
    Kiểm tra xem có người thắng theo các cột dọc hay không.
    """
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[0][i] # Lấy ô đầu tiên của cột làm mẫu
        for j in range(board_len):
            if board[j][i] != winner_val: # So sánh các ô trong cột đó
                winner_val = None
        if winner_val:
            return winner_val
    return winner_val

# Hàm kiểm tra thắng theo đường chéo
def get_diagonal_winner(board):
    """
    Kiểm tra xem có người thắng theo hai đường chéo (chính và phụ).
    """
    winner_val = None
    board_len = len(board)

    # 1. Kiểm tra đường chéo chính
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
    if winner_val:
        return winner_val

    # 2. Kiểm tra đường chéo phụ
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None
    return winner_val

# Hàm xác định người thắng chung cuộc
def winner(board):
    """
    Trả về người thắng cuộc của trò chơi (nếu có).
    Kết hợp kết quả từ kiểm tra Ngang, Dọc và Chéo.
    """
    # Nếu tìm thấy người thắng ở bất kỳ hướng nào, trả về người đó. Ngược lại trả về None.
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
    return winner_val

# Hàm kiểm tra trạng thái kết thúc
def terminal(board):
    """
    Trả về True nếu trò chơi đã kết thúc, False nếu chưa.
    Trò chơi kết thúc khi:
    1. Đã có người thắng (winner != None).
    2. Hoặc bàn cờ đã đầy (Hòa).
    """
    # Trường hợp 1: Có người thắng
    if winner(board) != None:
        return True

    # Trường hợp 2: Kiểm tra bàn cờ còn ô trống không
    for i in board:
        for j in i:
            if j == EMPTY: # Nếu còn ít nhất 1 ô trống
                return False

    # Nếu không còn ô trống nào -> Hòa -> Kết thúc
    return True

# Hàm định giá
def utility(board):
    """
    Trả về giá trị lợi ích của trạng thái kết thúc.
    - Trả về 1 nếu X thắng.
    - Trả về -1 nếu O thắng.
    - Trả về 0 nếu Hòa.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

*Thuật toán Minimax*

---
Sử dụng đệ quy (maxValue và minValue) để tìm nước đi tối ưu nhất.


In [37]:
# Hàm tìm giá trị lớn nhất
def maxValue(state):
    """
    Trả về giá trị tiện ích lớn nhất có thể đạt được từ trạng thái này.
    """
    # 1. Điều kiện dừng: Nếu game kết thúc, trả về điểm số
    if terminal(state):
        return utility(state)

    # 2. Khởi tạo giá trị ban đầu là âm vô cùng (tệ nhất với MAX)
    v = -math.inf

    # 3. Duyệt qua các hành động, tìm giá trị lớn nhất từ các phản ứng của MIN
    for action in actions(state):
        v = max(v, minValue(result(state, action)))

    return v

# Hàm tìm giá trị nhỏ nhất
def minValue(state):
    """
    Trả về giá trị tiện ích nhỏ nhất có thể đạt được từ trạng thái này.
    """
    # 1. Điều kiện dừng
    if terminal(state):
        return utility(state)

    # 2. Khởi tạo giá trị ban đầu là dương vô cùng (tệ nhất với MIN)
    v = math.inf

    # 3. Duyệt qua các hành động, tìm giá trị nhỏ nhất từ các phản ứng của MAX
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))

    return v

# Hàm chính: Quyết định nước đi tối ưu
def minimax(board):
    """
    Trả về nước đi tối ưu nhất cho người chơi hiện tại.
    """
    current_player = player(board)

    # --- TRƯỜNG HỢP 1: Lượt của MAX ---
    if current_player == X:
        best_val = -math.inf
        best_move = None

        # Duyệt tất cả các nước đi có thể
        for action in actions(board):
            # Giả định đi nước này, rồi xem MIN sẽ làm gì (gọi minValue)
            val = minValue(result(board, action))

            # Nếu tìm thấy nước đi có giá trị cao hơn giá trị tốt nhất hiện tại
            if val > best_val:
                best_val = val
                best_move = action

        return best_move

    # --- TRƯỜNG HỢP 2: Lượt của MIN ---
    else:
        best_val = math.inf
        best_move = None

        # Duyệt tất cả các nước đi có thể
        for action in actions(board):
            # Giả định đi nước này, rồi xem MAX sẽ làm gì (gọi maxValue)
            val = maxValue(result(board, action))

            # Nếu tìm thấy nước đi có giá trị thấp hơn (tốt cho MIN)
            if val < best_val:
                best_val = val
                best_move = action

        return best_move

*Chương trình chính*

In [ ]:
if __name__ == "__main__":
    # Khởi tạo bàn cờ
    n = int(input("Enter the size of the board: "))
    board = initial_state(n)
    ai_turn = False

    print("Choose a player (X or O):")
    user = input()

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            winner_is = winner(board)
            if winner_is is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {winner_is} wins.")
            break

        else:
            if user != playr and not game_over:
                if ai_turn:
                    print("AI is thinking...")
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    print(numpy.array(board))

            elif user == playr and not game_over:
                ai_turn = True
                print(numpy.array(board))
                print("Enter the position to move (row,col)")
                try:
                    i = int(input(f"Row (0-{n-1}): "))
                    j = int(input(f"Col (0-{n-1}): "))
                    if board[i][j] == EMPTY:
                        board = result(board, (i, j))
                        print(numpy.array(board))
                    else:
                        print("Spot already taken!")
                        ai_turn = False # Cho nhập lại
                except (ValueError, IndexError):
                    print("Invalid input! Please enter numbers 0-2.")
                    ai_turn = False

### ALPHA-BETA PRUNING

*1. Định nghĩa và mục đích*

---



- Alpha-Beta Pruning là một kỹ thuật tối ưu hóa được áp dụng cho thuật toán Minimax. Nó vận hành bằng cách duy trì hai giá trị đại diện ($\alpha, \beta$) trong quá trình duyệt cây để loại bỏ các nhánh không có khả năng ảnh hưởng đến quyết định cuối cùng.
- Mục đích cốt lõi của phương pháp là giải quyết vấn đề "bùng nổ tổ hợp" của Minimax, giúp giảm bớt không gian tìm kiếm và cho phép thuật toán tính toán ở độ sâu (depth) lớn hơn trong cùng một khoảng thời gian.

*2. Cơ chế hoạt động*

---

Quá trình thực thi của Alpha-Beta Pruning có thể được mô hình hóa qua 3 nguyên tắc tuần tự:


1. **Khởi tạo và Duy trì tham số**: Thuật toán duy trì hai giá trị xuyên suốt quá trình duyệt cây:
- **Alpha ($\alpha$)**: Giá trị tốt nhất (lớn nhất) mà người chơi MAX đã tìm thấy cho đến thời điểm hiện tại.
- **Beta ($\beta$)**: Giá trị tốt nhất (nhỏ nhất) mà người chơi MIN đã tìm thấy cho đến thời điểm hiện tại.
2. **Điều kiện cắt tỉa (Pruning Condition)**: Quá trình tìm kiếm tại một nhánh con sẽ dừng lại ngay lập tức (cắt bỏ nhánh) khi điều kiện $\alpha \geq \beta$ xảy ra. Điều này ngụ ý rằng phương án hiện tại đã tồi tệ hơn một phương án đã được tìm thấy trước đó, nên không cần xét tiếp.
3. **Cập nhật giá trị (Value Update):**
- **Tại lớp MAX**: Cập nhật $\alpha = \max(\alpha, \text{giá trị con})$.
- **Tại lớp MIN**: Cập nhật $\beta = \min(\beta, \text{giá trị con})$.

*3. Ví dụ minh họa - Trò chơi cờ caro (TicTacToe)*

---
Trong ngữ cảnh ứng dụng cho Tic-Tac-Toe (và các game đối kháng nói chung), kỹ thuật này đảm bảo kết quả tìm được là tối ưu giống hệt Minimax nhưng với hiệu suất cao hơn.
- **Chiến lược Thắng nhanh - Thua chậm**: Mã nguồn cài đặt sử dụng độ sâu (depth) làm trọng số trong hàm lượng giá. Thắng ở độ sâu thấp (+10 - depth) được ưu tiên hơn thắng ở độ sâu cao; ngược lại, thua ở độ sâu cao (-10 + depth) được chấp nhận hơn thua ngay lập tức.
- **Hiệu quả tìm kiếm**: Với Tic-Tac-Toe, Alpha-Beta có thể cắt giảm đáng kể số lượng node phải duyệt, đặc biệt ở các nước đi đầu tiên khi cây trò chơi còn rất rộng.


*Khởi tạo và Các hàm tiện ích*

In [55]:
import math
import copy

# X, O, EMPTY, user, ai are assumed to be declared globally by the main script and previous cells.
# The 'initial_state' function is also defined in a previous cell and used for board initialization.
# The functions 'PrintBoard' and 'GetAvailableCells' in the original Alpha-Beta section were designed
# for a 1D board representation and are not compatible with the 2D board used here. We will rely
# on 'actions(board)' from the Minimax section for getting available moves and standard Python printing for the board.

*Hàm định giá và Kiểm tra kết thúc*

---

Xác định trạng thái thắng/thua/hòa của bàn cờ dựa trên luật chơi (ngang, dọc, chéo).

In [56]:
# The 'GetWinner' function in the original Alpha-Beta section was designed for a 1D board representation.
# We will rely on the 'winner(board)' function defined in the Minimax section (cell GeR-dw1QE0F1)
# which is compatible with the 2D board used in the main program.

*Thuật toán Alpha-Beta*

---

Hàm đệ quy thực hiện việc duyệt cây, tính điểm và cắt tỉa nhánh thừa.

In [57]:
# This function implements the recursive Alpha-Beta pruning logic.
# It assumes X, O, EMPTY, terminal, utility, actions, and result are accessible from earlier cells.

def _alpha_beta_recursive(board, depth, alpha, beta, is_max_player):
    # Base case: if game is over, return the utility (score) of the state.
    # Optionally, include depth in the utility to prefer faster wins or slower losses.
    if terminal(board):
        score = utility(board)
        if score == 1:  # MAX player wins
            return score + (10 - depth) # Prefer winning faster
        elif score == -1: # MIN player wins
            return score - (10 - depth) # Prefer making MAX lose slower
        else: # Tie
            return score

    # If it's the MAX player's turn (e.g., 'X')
    if is_max_player:
        max_eval = -math.inf
        # Iterate through all possible moves using the 'actions' function for 2D boards.
        for action in actions(board):
            new_board = result(board, action) # Get the new board state after taking the action.
            # Recursively call for the MIN player (is_max_player=False).
            eval = _alpha_beta_recursive(new_board, depth + 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval) # Update alpha (best value found for MAX so far)
            if beta <= alpha: # Alpha-Beta Pruning condition: If current MAX move is already worse than MIN's best option, prune.
                break
        return max_eval
    # If it's the MIN player's turn (e.g., 'O')
    else: # is_max_player is False
        min_eval = math.inf
        # Iterate through all possible moves using the 'actions' function for 2D boards.
        for action in actions(board):
            new_board = result(board, action) # Get the new board state after taking the action.
            # Recursively call for the MAX player (is_max_player=True).
            eval = _alpha_beta_recursive(new_board, depth + 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval) # Update beta (best value found for MIN so far)
            if beta <= alpha: # Alpha-Beta Pruning condition: If current MIN move is already better than MAX's best option, prune.
                break
        return min_eval

*Hàm quyết định*

In [58]:
# This function determines the best move for the AI player using Alpha-Beta Pruning.
# It assumes X, O, player, actions, result, and _alpha_beta_recursive are accessible.

def FindBestMove(board, ai_player_char):
    """
    Tìm nước đi tối ưu nhất cho người chơi AI bằng Alpha-Beta Pruning.
    Sử dụng board 2D và các hàm game logic từ phần Minimax.
    """
    best_move = None
    current_ai_player = ai_player_char

    # If AI is the MAX player (e.g., 'X')
    if current_ai_player == X:
        best_val = -math.inf
        # Iterate through all possible actions
        for action in actions(board):
            new_board = result(board, action) # Simulate AI making this move
            # Call the recursive alpha-beta function, the next turn will be for the MIN player (opponent)
            move_val = _alpha_beta_recursive(new_board, 0, -math.inf, math.inf, False)
            if move_val > best_val:
                best_val = move_val
                best_move = action
    # If AI is the MIN player (e.g., 'O')
    else: # current_ai_player == O
        best_val = math.inf
        # Iterate through all possible actions
        for action in actions(board):
            new_board = result(board, action) # Simulate AI making this move
            # Call the recursive alpha-beta function, the next turn will be for the MAX player (opponent)
            move_val = _alpha_beta_recursive(new_board, 0, -math.inf, math.inf, True)
            if move_val < best_val:
                best_val = move_val
                best_move = action

    return best_move

*Chương trình chính*

In [ ]:
import numpy as np
if __name__ == "__main__":
    n = int(input("Enter the size of the board (e.g., 3): "))

    board = initial_state(n)

    print("Choose a player (X or O):")
    while True:
        user = input().strip().upper()
        if user in ["X", "O"]:
            break
        print("Invalid choice. Please enter X or O.")

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    print(f"\nGame Start! User: {user} vs AI: {ai}")
    print(np.array(board))

    while True:
        game_over = terminal(board)
        current_player = player(board)

        if game_over:
            print("\n-------------------")
            print(np.array(board))
            winner_is = winner(board)
            if winner_is is None:
                print(">>> Game Over: Tie.")
            else:
                print(f">>> Game Over: {winner_is} wins.")
            break

        if current_player != user:
            print(f"\n[{ai}] AI is thinking...")

            # Corrected: Call FindBestMove for Alpha-Beta pruning
            move = FindBestMove(board, ai)

            if move is None:
                print("AI cannot find a move (Error).")
                break

            board = result(board, move)
            print(f"AI chose: {move}")
            print(np.array(board))

        else:
            print(f"\n[{user}] Your turn.")
            while True:
                try:
                    i = int(input(f"Row (0-{n-1}): "))
                    j = int(input(f"Col (0-{n-1}): "))

                    if 0 <= i < n and 0 <= j < n:
                        if board[i][j] == EMPTY:
                            board = result(board, (i, j))
                            print(np.array(board))
                            break
                        else:
                            print("Spot already taken! Try again.")
                    else:
                        print(f"Coordinates must be between 0 and {n-1}.")

                except ValueError:
                    print("Invalid input! Please enter integers.")

### SO SÁNH

### SO SÁNH THUẬT TOÁN MINIMAX VÀ ALPHA-BETA PRUNING


---


#### 1. Điểm giống nhau

*   **Mục đích**: Cả hai thuật toán đều được sử dụng để tìm nước đi tối ưu trong các trò chơi đối kháng hai người (zero-sum games), đảm bảo người chơi có thể đưa ra quyết định tốt nhất dựa trên các trạng thái có thể xảy ra trong tương lai của trò chơi.
*   **Cơ sở lý thuyết**: Alpha-Beta Pruning là một phiên bản tối ưu hóa của Minimax. Nó không thay đổi kết quả cuối cùng mà Minimax tìm được; thay vào đó, nó đạt được cùng một kết quả nhưng với hiệu suất cao hơn.
*   **Sử dụng hàm định giá**: Cả hai đều dựa vào một hàm định giá (utility function) để đánh giá các trạng thái kết thúc (terminal states) hoặc các trạng thái trung gian (trong trường hợp tìm kiếm giới hạn độ sâu).
*   **Duyệt cây**: Cả hai đều duyệt qua cây trò chơi (game tree) để đánh giá các nước đi tiềm năng.

#### 2. Điểm khác nhau

| Tiêu chí                 | Minimax                                                                    | Alpha-Beta Pruning                                                       |
| :----------------------- | :------------------------------------------------------------------------- | :----------------------------------------------------------------------- |
| **Cơ chế hoạt động**     | Duyệt toàn bộ cây trò chơi theo chiều sâu (DFS), đánh giá tất cả các nút. | Duyệt cây theo chiều sâu nhưng loại bỏ (cắt tỉa) các nhánh không cần thiết mà không ảnh hưởng đến kết quả cuối cùng. |
| **Hiệu suất**            | Kém hiệu quả hơn, đặc biệt với các cây trò chơi lớn vì phải duyệt mọi khả năng. | Hiệu quả hơn đáng kể, cho phép tìm kiếm sâu hơn trong cùng một khoảng thời gian do giảm số lượng nút cần duyệt. |
| **Tham số bổ sung**      | Không có.                                                                  | Sử dụng hai tham số `alpha` (giá trị tốt nhất cho MAX) và `beta` (giá trị tốt nhất cho MIN) để thực hiện cắt tỉa. |
| **Điều kiện cắt tỉa**    | Không có. Luôn duyệt tất cả các nhánh.                                     | Cắt tỉa khi `beta <= alpha`, tức là khi một nhánh đã trở nên tệ hơn một phương án đã được tìm thấy. |
| **Độ phức tạp**          | Độ phức tạp thời gian và không gian là $O(b^d)$ (với $b$ là hệ số phân nhánh, $d$ là độ sâu). | Độ phức tạp thời gian tốt nhất là $O(b^{d/2})$, giảm đáng kể so với Minimax. |
| **Ứng dụng thực tế**     | Hạn chế hơn với các trò chơi phức tạp do yêu cầu tính toán lớn.             | Được ưa chuộng hơn trong các hệ thống AI chơi game thực tế do hiệu quả vượt trội. |